In [ ]:
!sudo apt update
!sudo apt install -y libcairo2-dev libpango1.0-dev ffmpeg
# Install a stable version of Manim that works in Colab
!pip install manim==0.17.3
!pip install pydantic
#!pip install pydantic==1.10.7
!pip install typing-extensions==4.5.0
!pip install typing

!pip install flask-ngrok

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
37 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
    resolver = self.make_resolver(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 177, in make_resolver
    return pip._internal.resolution.resolvelib.resolver.Resolver(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 58, in __init__
    self.factory = Factory(
                   ^^^^^^^^
  File "/usr/local/lib/py

In [ ]:
# Load the Manim extension to use %%manim magic
%load_ext manim

The manim module is not an IPython extension.


In [ ]:
from pydantic import BaseModel
from typing import List, Optional

class Unit(BaseModel):
    id: str
    name: str
    type: str  # e.g. "infantry", "armor", "air support"
    strength: int  # combat effectiveness, 0–100
    position: Optional[tuple[float, float]] = None  # (x, y) coords
    allegiance: str  # "friendly" or "enemy"
    status: str = "active"  # e.g. "active", "retreating", "destroyed"

class TerrainFeature(BaseModel):
    type: str  # e.g. "hill", "forest", "building"
    position: tuple[float, float]
    size: float  # area/radius in meters

class Terrain(BaseModel):
    type: str  # e.g. "urban", "desert", "jungle"
    features: List[TerrainFeature]
    dimensions: tuple[int, int]  # map size in meters (width, height)

class Objective(BaseModel):
    id: str
    description: str
    controlling_unit_ids: List[str] = []
    completed: bool = False
    location: Optional[tuple[float, float]]
    priority: int = 1  # Higher number = more critical

class BattleEvent(BaseModel):
    timestamp: object  # e.g. "00:05", "12:03 PM"
    description: str
    involved_units: List[str] = []
    event_type: str  # e.g. "move", "fire", "retreat", "reinforce"
    location: Optional[tuple[float, float]]

class Scenario(BaseModel):
    title: str
    description: str
    terrain: Terrain
    units: List[Unit]
    objectives: List[Objective]
    timeline: List[BattleEvent]

In [ ]:
lunar_input = """
        Title: LUNAR ASCENT
        Description: A covert laser strike has crippled UED’s high-bandwidth relay satellite over the Moon. Tensions escalate to open orbital warfare. The Moon becomes the fulcrum.
        Unit:ID=U1, Name=UED Defense Satellite 1, Type=satellite, Strength=70, Allegiance=friendly, X=-4, Y=2
        Unit:ID=U2, Name=UED Defense Satellite 2, Type=satellite, Strength=70, Allegiance=home, X=-3, Y=2.5
        Unit:ID=U3, Name=UED Supply Shuttle, Type=shuttle, Strength=100, Allegiance=home, X=-2, Y=-1
        Unit:ID=U4, Name=UED Artemis Laser, Type=ground-laser, Strength=90, Allegiance=friendly, X=0, Y=-2
        Unit:ID=U5, Name=CC Stealth Satellite 1, Type=stealth-satellite, Strength=60, Allegiance=enemy, X=3, Y=2.5
        Unit:ID=U6, Name=CC Stealth Satellite 2, Type=stealth-satellite, Strength=60, Allegiance=enemy, X=4, Y=2
        Unit:ID=U7, Name=CC Lunar Jammer, Type=jammer, Strength=50, Allegiance=enemy, X=1, Y=-2
        Unit:ID=U8, Name=CC Drone Swarm, Type=drone, Strength=80, Allegiance=enemy, X=2, Y=1
        Objective:ID=O1, Desc=Retake orbital sectors, X=0, Y=2, Priority=1
        Objective:ID=O2, Desc=Protect Artemis Base, X=0, Y=-2, Priority=2
        Event:Time=Turn 1, Desc=Deploy UED Defense Satellites, Units=U1|U2, Type=move
        Event:Time=Turn 2, Desc=CC launches jammer activation, Units=U7, Type=hold
        Event:Time=Turn 3, Desc=Direct fire between satellites begins, Units=U1|U5, Type=fire
        Event:Time=Turn 4, Desc=UED Artemis Laser engages drones, Units=U4|U8, Type=fire
        Event:Time=Turn 5, Desc=CC Stealth Satellite attempts repositioning, Units=U6, Type=move
        Event:Time=Turn 6, Desc=UED Supply Shuttle prepares for descent, Units=U3, Type=hold
        Event:Time=Turn 7, Desc=UED reclaims orbital sector, Units=U2, Type=move
        """


dynamo_input="""
        Title:Operation Dynamo
        Description:Allied forces are retreating and preparing for naval evacuation under fire from advancing Axis troops.
        Unit:ID=U1, Name=British Infantry, Type=infantry, Strength=85, Allegiance=friendly, X=-3, Y=-2.5
        Unit:ID=U2, Name=French Infantry, Type=infantry, Strength=78, Allegiance=friendly, X=-1, Y=-2.2
        Unit:ID=U3, Name=German Armor, Type=armor, Strength=92, Allegiance=enemy, X=2, Y=-1.8
        Feature:Type=Bunker, X=0, Y=-2, Size=10
        Objective:ID=O1, Desc=Evacuate to naval boats, X=4, Y=0.5, Priority=1
        Event:Time=00:00, Desc=British Infantry begins fallback to coast, Units=U1, Type=move
        Event:Time=00:01, Desc=German Armor fires on beach, Units=U3, Type=fire
        Event:Time=00:02, Desc=French Infantry holds position, Units=U2, Type=hold
        """

#Lunar based scenario

In [ ]:
#from models import *

# ------------------------
# Parser Function
# ------------------------

def parse_scenario_text(text: str) -> Scenario:
    lines = [line.strip() for line in text.strip().split("\n") if line.strip()]
    units = []
    features = []
    objectives = []
    timeline = []
    title = "Generated Scenario"
    description = ""
    terrain_type = "beach"
    terrain_dims = (1000, 1000)

    for line in lines:
        if line.startswith("Title:"):
            title = line.split(":", 1)[1].strip()
        elif line.startswith("Description:"):
            description = line.split(":", 1)[1].strip()
        elif line.startswith("Unit:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            print(props)
            unit = Unit(
                id=props["ID"],
                name=props["Name"],
                type=props["Type"],
                strength=int(props["Strength"]),
                allegiance=props["Allegiance"],
                position=(float(props.get("X", 0)), float(props.get("Y", 0)))
            )
            units.append(unit)
        elif line.startswith("Feature:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            features.append(TerrainFeature(
                type=props["Type"],
                position=(float(props["X"]), float(props["Y"])),
                size=float(props["Size"])
            ))
        elif line.startswith("Objective:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            objectives.append(Objective(
                id=props["ID"],
                description=props["Desc"],
                location=(float(props.get("X", 0)), float(props.get("Y", 0))),
                priority=int(props.get("Priority", 1))
            ))
        elif line.startswith("Event:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            timeline.append(BattleEvent(
                timestamp=props["Time"],
                description=props["Desc"],
                involved_units=props["Units"].split("|"),
                event_type=props["Type"]
            ))

    return Scenario(
        title=title,
        description=description,
        terrain=Terrain(type=terrain_type, features=features, dimensions=terrain_dims),
        units=units,
        objectives=objectives,
        timeline=timeline
    )

Scene_it = parse_scenario_text(test_input)

{'ID': 'U1', 'Name': 'UED Defense Satellite 1', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '-4', 'Y': '2'}
{'ID': 'U2', 'Name': 'UED Defense Satellite 2', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '-3', 'Y': '2.5'}
{'ID': 'U3', 'Name': 'UED Defense Satellite 3', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '-2', 'Y': '3'}
{'ID': 'U4', 'Name': 'UED Defense Satellite 4', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '-1', 'Y': '3.2'}
{'ID': 'U5', 'Name': 'UED Defense Satellite 5', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '0', 'Y': '2.8'}
{'ID': 'U6', 'Name': 'UED Defense Satellite 6', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '1', 'Y': '2.5'}
{'ID': 'U7', 'Name': 'UED Defense Satellite 7', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '2', 'Y': '2'}
{'ID': 'U8', 'Name': 'UED Defense Satellite 8', 'Type': '

In [ ]:
%%manim -qm DunkirkBeachScene

from manim import *

class DunkirkBeachScene(Scene):
    def construct(self):

        # Set white background
        background = Rectangle(width=20, height=12, fill_color=BLACK, fill_opacity=1).set_z_index(-1)
        self.add(background)

        # Load SVG representing France map
        france_svg = SVGMobject("sample_data/nord-france.svg")
        france_svg.scale(2.5)
        france_svg.set_fill(YELLOW_E, opacity=0.3)
        france_svg.set_stroke(YELLOW_E)
        self.add(france_svg)

        # Title
        title = Text("Dunkirk Evacuation - Visualization").scale(0.6).to_edge(UP)
        self.add(title)

        # Units - manually placed on Dunkirk coastal region approximation
        british_troop = Dot(point=[1, 2, 0], radius=0.15, color=GREEN)
        british_label = Text("British Infantry").scale(0.4).next_to(british_troop, UP)

        french_troop = Dot(point=[1, 1.5, 0], radius=0.15, color=BLUE)
        french_label = Text("French Infantry").scale(0.4).next_to(french_troop, UP)

        german_troop = Dot(point=[0.5, 1, 0], radius=0.15, color=RED)
        german_label = Text("German Forces").scale(0.4).next_to(german_troop, UP)

        self.add(british_troop, british_label, french_troop, french_label, german_troop, german_label)

        # Evacuation Zone
        evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([2, 2, 0])
        evac_label = Text("Evacuation Zone").scale(0.3).next_to(evac_zone, DOWN)
        self.add(evac_zone, evac_label)

        # Animate British movement to evacuation zone
        self.wait(1)
        self.play(british_troop.animate.move_to([2, 2, 0]), run_time=3)
        self.wait(1)

        # Final message
        complete_label = Text("Evacuation Successful").scale(0.5).next_to(evac_zone, UP).set_color(GREEN)
        self.play(FadeIn(complete_label))
        self.wait(2)


[04/06/25 05:19:21] INFO     Animation 0 : Partial movie file written in                   ]8;id=761523;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=782726;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/274514146_1076466279_620829335.mp4'                                    

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/274514146_1076466279_620829335.mp4'


[04/06/25 05:19:25] INFO     Animation 1 : Partial movie file written in                   ]8;id=579723;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=109709;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_2669002150_1871908949.mp4'                                  

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/2789044632_2669002150_1871908949.mp4'


[04/06/25 05:19:27] INFO     Animation 2 : Partial movie file written in                   ]8;id=238443;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=766828;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_345352296_1634755845.mp4'                                   

...

INFO:manim:Combining to Movie file.


                    INFO                                                                   ]8;id=533145;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=749851;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'                          
                                                                                                                   

INFO:manim:
File ready at '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'



                    INFO     Rendered DunkirkBeachScene                                                ]8;id=397848;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=787162;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 5 animations                                                                   

INFO:manim:Rendered DunkirkBeachScene
Played 5 animations


In [ ]:
### Below is an example with shapes verse a map


In [ ]:
%%manim -qm DunkirkBeachScene

from manim import *
class DunkirkBeachScene(Scene):
    def construct(self):
        # Draw stylized beach and water using shapes
        beach = Rectangle(width=12, height=4, fill_color=YELLOW_E, fill_opacity=0.4).move_to([0, -2, 0])
        sea = Rectangle(width=12, height=3, fill_color=BLUE, fill_opacity=0.3).next_to(beach, UP, buff=0)
        self.add(sea, beach)

        # Title
        title = Text("Dunkirk Evacuation - Visualization").scale(0.6).to_edge(UP)
        self.add(title)

        # Units - positioned on beach
        british_troop = Dot(point=[-3, -2.5, 0], radius=0.15, color=GREEN)
        british_label = Text("British Infantry").scale(0.4).next_to(british_troop, UP)

        french_troop = Dot(point=[-1, -2.2, 0], radius=0.15, color=BLUE)
        french_label = Text("French Infantry").scale(0.4).next_to(french_troop, UP)

        german_troop = Dot(point=[2, -1.8, 0], radius=0.15, color=RED)
        german_label = Text("German Forces").scale(0.4).next_to(german_troop, UP)

        self.add(british_troop, british_label, french_troop, french_label, german_troop, german_label)

        # Evacuation Zone (placed in the sea)
        evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([4, 0.5, 0])
        evac_label = Text("Evacuation Zone").scale(0.3).next_to(evac_zone, DOWN)
        self.add(evac_zone, evac_label)

        # Animate British movement to evacuation zone
        self.wait(1)
        self.play(british_troop.animate.move_to([4, 0.5, 0]), run_time=3)
        self.wait(1)

        # Final message
        complete_label = Text("Evacuation Successful").scale(0.5).next_to(evac_zone, UP).set_color(GREEN)
        self.play(FadeIn(complete_label))
        self.wait(2)


[04/05/25 16:40:31] INFO     Animation 0 : Partial movie file written in                   ]8;id=252612;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=323568;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/274514146_1076466279_140515962.mp4'                                    

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/274514146_1076466279_140515962.mp4'


[04/05/25 16:40:36] INFO     Animation 1 : Partial movie file written in                   ]8;id=239740;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=505066;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_445026133_1359018374.mp4'                                   

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/2789044632_445026133_1359018374.mp4'


[04/05/25 16:40:37] INFO     Animation 2 : Partial movie file written in                   ]8;id=740743;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=423864;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_345352296_1223585236.mp4'                                   

...

INFO:manim:Combining to Movie file.


                    INFO                                                                   ]8;id=533519;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=465952;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'                          
                                                                                                                   

INFO:manim:
File ready at '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'



                    INFO     Rendered DunkirkBeachScene                                                ]8;id=611154;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=541420;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 5 animations                                                                   

INFO:manim:Rendered DunkirkBeachScene
Played 5 animations


# Dynamic JSON-Driven Manim Script

In [ ]:
%%manim -qm ScenarioScene

import json
from manim import *

class ScenarioScene(Scene):
    def construct(self):
        # Load scenario data
        with open("space_scenario.json", "r") as f:
            data = json.load(f)

        self.unit_mobs = {}

        # Create the grid
        grid = NumberPlane(
            x_range=[-1, data["terrain"]["dimensions"][0] + 1, 1],
            y_range=[-1, data["terrain"]["dimensions"][1] + 2, 1],
            background_line_style={"stroke_opacity": 0.2}
        ).scale(0.6)
        self.add(grid)

        # Title
        title = Text(data["title"].strip("."), font_size=32).to_edge(UP)
        self.play(Write(title))


        # Terrain color mapping
        TERRAIN_COLORS = {
            "Beach": LIGHT_BROWN,
            "Forest": GREEN,
            "Mountain": GRAY,
            "River": BLUE,
            "Urban": DARK_GRAY,
            "Default": LIGHT_GRAY
        }
        # --- Terrain features ---
        for feature in data["terrain"]["features"]:
          t_type = feature.get("type", "Default")
          color = TERRAIN_COLORS.get(t_type, TERRAIN_COLORS["Default"])

          # You can customize shape based on type too, but using Rectangle for now
          terrain_shape = Rectangle(width=2, height=1, color=color, fill_opacity=0.3)
          terrain_shape.move_to(grid.c2p(*feature["position"]))

          label = Text(t_type, font_size=16).next_to(terrain_shape, UP, buff=0.1)
          self.play(FadeIn(terrain_shape), Write(label))

        # --- Objectives ---
        for obj in data["objectives"]:
            dot = Dot(grid.c2p(*obj["location"]), color=ORANGE)
            label = Text(obj["description"], font_size=16).next_to(dot, RIGHT, buff=0.1)
            self.play(FadeIn(dot), Write(label))

        # --- Units ---
        for unit in data["units"]:
            pos = unit["position"]
            color = BLUE if unit["allegiance"] == "Friendly" else RED
            unit_circle = Circle(radius=0.25, color=color, fill_opacity=0.6)
            unit_circle.move_to(grid.c2p(*pos))
            label = Text(unit["id"], font_size=14).move_to(unit_circle.get_center())
            unit_group = VGroup(unit_circle, label)
            self.play(FadeIn(unit_group), run_time=0.3)
            self.unit_mobs[unit["id"]] = unit_group

        # --- Timeline Animation w/ explicit location ---
        self.wait(0.5)
        for event in data["timeline"]:
            desc = event["description"]
            display_text = Text(desc, font_size=20).to_edge(DOWN)
            self.play(Write(display_text), run_time=0.5)

            for uid in event["involved_units"]:
                mob = self.unit_mobs.get(uid)
                if not mob:
                    continue

                self.play(Indicate(mob), run_time=0.4)

                # Move if event has a location
                if event.get("location"):
                    dest_point = grid.c2p(*event["location"])
                    self.play(mob.animate.move_to(dest_point), run_time=1)

                # Optional: simple animation for "fire" event type
                if event.get("event_type") == "fire" and event.get("location"):
                  firing_pos = mob.get_center()
                  target_pos = grid.c2p(*event["location"])

                  # Create the line and fire dot
                  line = Line(firing_pos, target_pos, color=WHITE)
                  fire_dot = Dot(color=YELLOW).move_to(firing_pos)

                  self.add(line, fire_dot)
                  self.play(MoveAlongPath(fire_dot, line), run_time=0.6)
                  self.remove(fire_dot, line)

            self.remove(display_text)
            self.wait(0.3)

        self.wait(2)

[04/08/25 12:00:36] INFO     Animation 0 : Partial movie file written in                   ]8;id=609632;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=190788;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Sce                         
                             narioScene/274514146_1362341462_731523005.mp4'                                        

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/ScenarioScene/274514146_1362341462_731523005.mp4'


[04/08/25 12:00:38] INFO     Animation 1 : Partial movie file written in                   ]8;id=434978;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=262464;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Sce                         
                             narioScene/2789044632_2465253442_2091747051.mp4'                                      

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/ScenarioScene/2789044632_2465253442_2091747051.mp4'


[04/08/25 12:00:41] INFO     Animation 2 : Partial movie file written in                   ]8;id=629066;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=861281;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Sce                         
                             narioScene/2789044632_988387346_1337386828.mp4'                                       

...

INFO:manim:
File ready at '/content/media/videos/content/720p30/ScenarioScene.mp4'



                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=429355;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=37421;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#708\708]8;;\
                             manim has removed the 34 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

INFO:manim:The partial movie directory is full (> 100 files). Therefore, manim has removed the 34 oldest file(s). You can change this behaviour by changing max_files_cached in config.


                    INFO     Rendered ScenarioScene                                                    ]8;id=348424;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=219591;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 34 animations                                                                  

INFO:manim:Rendered ScenarioScene
Played 34 animations


In [34]:
%%manim -qm SpaceScenario3DScene

import json
from manim import *

class SpaceScenario3DScene(ThreeDScene):
    def construct(self):
        # Load JSON data
        with open("space_scenario.json", "r") as f:
            data = json.load(f)

        self.unit_mobs = {}

        # Setup 3D camera
        self.set_camera_orientation(phi=70 * DEGREES, theta=45 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.02)

        # Draw Earth
        earth = Sphere(radius=1, color=BLUE, resolution=(24, 48))
        earth.set_fill(BLUE_E, opacity=0.5)
        earth.set_shade_in_3d(True)
        earth_label = Text("Earth", font_size=24).move_to([0, -1.4, 1.2]).scale(0.4)
        self.play(FadeIn(earth), Write(earth_label))

        self.add(earth)

        # --- Units ---
        for unit in data["units"]:
            x, y = unit["position"]
            z = 0.5  # Default Z height for orbit
            color = self.get_color(unit["allegiance"])
            dot = Dot3D(point=[x, y, z], radius=0.15, color=color)
            # Use 2D text and project it in 3D space relative to the dot
            label = always_redraw(lambda: Text( unit["id"], font_size=30).scale(0.9).move_to(dot.get_center() + UP * 0.3))
            group = VGroup(dot, label)
            self.add(group)
            self.unit_mobs[unit["id"]] = group

        # --- Objectives ---
        for obj in data.get("objectives", []):
            x, y = obj["location"]
            z = 0.5
            marker = Dot3D(point=[x, y, z], color=YELLOW)
            label = always_redraw(lambda: Text(obj["description"], font_size=12).scale(0.4).move_to(marker.get_center() + RIGHT * 0.4))
            self.add(marker, label)
            self.play(FadeIn(marker), Write(label))

        # --- Timeline ---
        for event in data.get("timeline", []):
            desc_text = Text(event["description"], font_size=50).to_edge(DOWN).scale(0.4)
            self.play(Write(desc_text), run_time=0.4)

            for uid in event["involved_units"]:
                mob = self.unit_mobs.get(uid)
                if not mob:
                    continue
                self.play(Indicate(mob), run_time=0.3)

                # Movement
                if event.get("event_type") == "move" and event.get("location"):
                    x, y = event["location"]
                    z = 0.5
                    self.play(mob.animate.move_to([x, y, z]), run_time=1)

                # Fire effect
                elif event.get("event_type") == "fire" and event.get("location"):
                    start = mob.get_center()
                    end = [*event["location"], 0.5]
                    beam = Line3D(start, end, color=RED)
                    pulse = Dot3D(start, color=RED)
                    self.add(beam, pulse)
                    self.play(MoveAlongPath(pulse, beam), run_time=0.6)
                    self.remove(beam, pulse)

            self.remove(desc_text)
            self.wait(0.2)

        self.wait(2)

    def get_color(self, allegiance):
        return {
            "Friendly": BLUE,
            "Home": GREEN,
            "Enemy": RED
        }.get(allegiance, WHITE)

[04/08/25 16:50:43] INFO     Animation 0 : Using cached data (hash :                           ]8;id=166714;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=579692;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#78\78]8;;\
                             2782073684_4237001726_729892282)                                                      

INFO:manim:Animation 0 : Using cached data (hash : 2782073684_4237001726_729892282)


[04/08/25 16:52:50] INFO     Animation 1 : Partial movie file written in                   ]8;id=119700;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=212013;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Spa                         
                             ceScenario3DScene/765951063_390318141_1393544043.mp4'                                 

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/SpaceScenario3DScene/765951063_390318141_1393544043.mp4'


[04/08/25 16:55:14] INFO     Animation 2 : Partial movie file written in                   ]8;id=154389;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=792744;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Spa                         
                             ceScenario3DScene/4026279760_3286737488_187924883.mp4'                                

...

INFO:manim:
File ready at '/content/media/videos/content/720p30/SpaceScenario3DScene.mp4'



                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=803326;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=125672;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#708\708]8;;\
                             manim has removed the 16 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

INFO:manim:The partial movie directory is full (> 100 files). Therefore, manim has removed the 16 oldest file(s). You can change this behaviour by changing max_files_cached in config.


                    INFO     Rendered SpaceScenario3DScene                                             ]8;id=949080;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=975921;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 26 animations                                                                  

INFO:manim:Rendered SpaceScenario3DScene
Played 26 animations


In [ ]:
%%manim -qm SpaceConflictScene
from manim import *


class SpaceConflictScene(Scene):
    def construct(self):

        scenario = parse_scenario_text(lunar_input)

        # Earth in center
        earth = Circle(radius=1.5, color=BLUE, fill_opacity=0.4).move_to([0, 0, 0])
        earth.rotate_about_origin(PI / 2)  # optional tilt
        self.play(Rotate(earth, angle=2 * PI, about_point=ORIGIN, run_time=20), rate_func=lambda t: t)
        earth_label = Text("Earth").scale(0.4).next_to(earth, DOWN)
        self.add(earth, earth_label)

        title = Text(scenario.title).scale(0.6).to_edge(UP)
        self.add(title)

        unit_dots = {}
        for unit in scenario.units:
            if unit.allegiance.lower() == "friendly":
                color = BLUE
            elif unit.allegiance.lower() == "neutral":
                color = GREEN
            else:
                color = RED
            if unit.type == "infantry":
                dot = Dot(point=[unit.position[0], unit.position[1], 0], radius=0.15, color=color)
            elif unit.type == "armor":
                dot = Square(side_length=0.3, color=color).move_to([unit.position[0], unit.position[1], 0])
            else:
                dot = Dot(point=[unit.position[0], unit.position[1], 0], radius=0.15, color=color)
            label = Text(unit.name).scale(0.4).next_to(dot, UP)
            self.add(dot, label)
            unit_dots[unit.id] = dot

        if scenario.objectives:
            evac = scenario.objectives[0]
            evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([evac.location[0], evac.location[1], 0])
            evac_label = Text("Objective Zone").scale(0.3).next_to(evac_zone, DOWN)
            self.add(evac_zone, evac_label)

        self.wait(1)
        # Group events by turn
        events_by_turn = {}
        for event in scenario.timeline:
            turn = event.timestamp.strip()
            if turn not in events_by_turn:
                events_by_turn[turn] = []
            events_by_turn[turn].append(event)

        # Animate each turn as a separate scene
        for turn, events in events_by_turn.items():
            turn_title = Text(f"{turn} Actions").scale(0.5).to_edge(UP)
            self.play(FadeIn(turn_title))
            self.wait(0.5)

            for event in events:
                for uid in event.involved_units:
                    if uid in unit_dots:
                        dot = unit_dots[uid]
                        if event.event_type == "move":
                            self.play(dot.animate.move_to([evac.location[0], evac.location[1], 0]), run_time=2)
                        elif event.event_type == "fire":
                            self.play(Flash(dot), run_time=1)
                        elif event.event_type == "hold":
                            self.play(dot.animate.scale(1.2).scale(1 / 1.2), run_time=1)

            self.wait(0.5)
            self.play(FadeOut(turn_title))

        for uid, dot in unit_dots.items():
            unit = next((u for u in scenario.units if u.id == uid), None)
            if unit and unit.allegiance == "enemy" and "satellite" in unit.type:
                self.play(Flash(dot, color=RED), run_time=1)
                label = Text(unit.id).scale(0.3).next_to(dot, DOWN)
                self.add(label)

                orbit = Circle(radius=2.5, color=WHITE, stroke_opacity=0.3).move_to(earth.get_center())
                orbit.set_z_index(-1)
                self.add(orbit)

                self.play(Rotate(dot, angle=2 * PI, about_point=orbit.get_center()), run_time=4)

        self.wait(1)
        self.play(FadeOut(turn_title))


{'ID': 'U1', 'Name': 'UED Defense Satellite 1', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'friendly', 'X': '-4', 'Y': '2'}
{'ID': 'U2', 'Name': 'UED Defense Satellite 2', 'Type': 'satellite', 'Strength': '70', 'Allegiance': 'home', 'X': '-3', 'Y': '2.5'}
{'ID': 'U3', 'Name': 'UED Supply Shuttle', 'Type': 'shuttle', 'Strength': '100', 'Allegiance': 'home', 'X': '-2', 'Y': '-1'}
{'ID': 'U4', 'Name': 'UED Artemis Laser', 'Type': 'ground-laser', 'Strength': '90', 'Allegiance': 'friendly', 'X': '0', 'Y': '-2'}
{'ID': 'U5', 'Name': 'CC Stealth Satellite 1', 'Type': 'stealth-satellite', 'Strength': '60', 'Allegiance': 'enemy', 'X': '3', 'Y': '2.5'}
{'ID': 'U6', 'Name': 'CC Stealth Satellite 2', 'Type': 'stealth-satellite', 'Strength': '60', 'Allegiance': 'enemy', 'X': '4', 'Y': '2'}
{'ID': 'U7', 'Name': 'CC Lunar Jammer', 'Type': 'jammer', 'Strength': '50', 'Allegiance': 'enemy', 'X': '1', 'Y': '-2'}


[04/07/25 12:32:44] INFO     Animation 0 : Using cached data (hash :                           ]8;id=74835;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=117488;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#78\78]8;;\
                             274514146_1310598406_957341620)                                                       

INFO:manim:Animation 0 : Using cached data (hash : 274514146_1310598406_957341620)


[04/07/25 12:32:45] INFO     Animation 1 : Partial movie file written in                   ]8;id=985811;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=990543;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Spa                         
                             ceConflictScene/2789044632_345352296_1350782862.mp4'                                  

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/SpaceConflictScene/2789044632_345352296_1350782862.mp4'


...

INFO:manim:
File ready at '/content/media/videos/content/720p30/SpaceConflictScene.mp4'



                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=12278;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=133689;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#708\708]8;;\
                             manim has removed the 1 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

INFO:manim:The partial movie directory is full (> 100 files). Therefore, manim has removed the 1 oldest file(s). You can change this behaviour by changing max_files_cached in config.


                    INFO     Rendered SpaceConflictScene                                               ]8;id=470234;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=389001;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 45 animations                                                                  

INFO:manim:Rendered SpaceConflictScene
Played 45 animations


#Add the form to add the scenario

In [ ]:
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
from after_action_models import Unit, TerrainFeature, Objective, BattleEvent, Terrain, Scenario

app = Flask(__name__)
run_with_ngrok(app)  # Automatically tunnel to ngrok

html_form = '''<!DOCTYPE html>
<html>
<head><title>Scenario Input</title></head>
<body>
<h2>Enter Scenario</h2>
<form action="/submit" method="post">
  <label>Title: <input name="title"></label><br>
  <label>Description: <textarea name="description"></textarea></label><br>
  <label>Units:<br><textarea name="units" rows="5" cols="60"></textarea></label><br>
  <label>Features:<br><textarea name="features" rows="5" cols="60"></textarea></label><br>
  <label>Objectives:<br><textarea name="objectives" rows="3" cols="60"></textarea></label><br>
  <label>Timeline:<br><textarea name="timeline" rows="5" cols="60"></textarea></label><br>
  <button type="submit">Submit</button>
</form>
</body>
</html>'''

@app.route("/")
def home():
    return render_template_string(html_form)

@app.route("/submit", methods=["POST"])
def submit():
    form = request.form
    # Parse form into Scenario as in your backend...
    return jsonify({"message": "Form received", "title": form.get("title")})

app.run()


In [ ]:
from IPython.display import IFrame
IFrame(src="https://abcd1234.ngrok.io", width="100%", height=600)

In [ ]:
    try:
        from openai import OpenAI
        import os

        openai_api_key = os.getenv("OPENAI_API_KEY")
        if openai_api_key:
            import openai
            openai.api_key = openai_api_key

            prompt = f"""
            Create a Python script using Manim to visualize this battle scenario:
            Title: {scenario.title}
            Description: {scenario.description}
            Units: {[u.dict() for u in scenario.units]}
            Features: {[f.dict() for f in scenario.terrain.features]}
            Objectives: {[o.dict() for o in scenario.objectives]}
            Timeline: {[e.dict() for e in scenario.timeline]}
            """

            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a Manim expert."},
                    {"role": "user", "content": prompt}
                ]
            )
            generated_code = response['choices'][0]['message']['content']
            return jsonify({"scenario": scenario.dict(), "manim_code": generated_code})
        else:
            return jsonify({"scenario": scenario.dict(), "note": "No OPENAI_API_KEY found."})
    except Exception as e:
        return jsonify({"scenario": scenario.dict(), "error": str(e)})

SyntaxError: 'return' outside function (<ipython-input-1-3755f0255514>, line 28)

In [ ]:
from IPython.display import IFrame
IFrame(src="https://abcd1234.ngrok.io", width="100%", height=600)